Founders and Financials: Machine Learning Algorithms in Venture Capital by Viktor Lado Naess and Emrik Stål



# excluding gender


In [ ]:
# importing all data
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

spreadsheet = gc.open('final data')

worksheet = spreadsheet.worksheet('Sheet1')



data = worksheet.get_all_values()

import pandas as pd
df = pd.DataFrame(data[1:], columns=data[0])
import numpy as np


#making sure all names and datatypes match
df.loc[:,'More than one round'] = df['More than one round'].astype(int)
df['Share female founders'] = df['Share female founders'].replace('', np.nan)
df.loc[:,'Share female founders'] = df['Share female founders'].astype(float)
df.loc[:,'Distance from Stockholm'] = df['Distance from Stockholm'].astype(float)
df.loc[:,'Serial Founder'] = df['Serial Founder'].astype(int)
df.loc[:,'Branch Sector'] = df['Branch Sector'].astype(int)
df = df.rename(columns={'Final_Second_Round_Year': 'Final Second Round Year'})
df.loc[:,'Final Second Round Year'] = df['Final Second Round Year'].astype(int)
df = df.rename(columns={'Founded_on_Year': 'Founded on Year'})
df.loc[:,'Founded on Year'] = df['Founded on Year'].astype(int)
df.loc[:,'Quick Ratio'] = df['Quick Ratio'].astype(float)
df.loc[:,'Return on Equity'] = df['Return on Equity'].astype(float)

df.loc[:,'Net_sales'] = df['Net_sales'].astype(float)
df = df.rename(columns={'Net_sales': 'Net Sales'})

df.loc[:,'Net Sales Ratio'] = df['Net Sales Ratio'].astype(float)
df.loc[:,'avg'] = df['avg'].astype(float)
df.loc[:,'Cluster'] = df['Cluster'].astype(float)

random_seed =8

In [ ]:
df

In [ ]:
#dropping outliers larger than 3 standard deviations away
from scipy import stats

columns = ['Return on Equity', 'Quick Ratio', 'Net Sales Ratio']

outlier_indices = set()

for col in columns:
    z_scores = stats.zscore(df[col])

    outliers = df[abs(z_scores) > 3].index

    outlier_indices.update(outliers)

df = df.drop(index=outlier_indices)

df

In [ ]:
df.describe()


In [ ]:
df.dtypes

In [ ]:
#Making a more specific description
import pandas as pd

non_numerical_columns = ['Name', 'Location']

summary_df = pd.DataFrame(columns=['Column', 'Mode', 'Median', 'Mean', 'Q1', 'Q2', 'Q3', 'Q4'])

for column in df.columns:
    if column not in non_numerical_columns:

        mode = df[column].mode()[0]
        median = df[column].median()
        mean = df[column].mean()
        std=df[column].std()
        q1 = df[column].quantile(0.25)
        q2 = df[column].quantile(0.50)
        q3 = df[column].quantile(0.75)
        q4 = df[column].quantile(1.00)


        summary_df = summary_df.append({
            'Column': column,
            'Mode': mode,
            'Median': median,
            'Mean': mean,
            'Stan dev.': std,
            'Q1': q1,
            'Q2': q2,
            'Q3': q3,
            'Q4': q4
        }, ignore_index=True)


summary_df


In [ ]:
#Getting Different city counts
import pandas as pd

categorical_column = 'Location'

mode_value = df[categorical_column].mode()[0]


value_counts = df[categorical_column].value_counts()


total_counts = df[categorical_column].count()
percentages = (value_counts / total_counts) * 100


categorical_counts_df = pd.DataFrame({'Counts': value_counts, 'Percentages': percentages})


categorical_counts_df = categorical_counts_df.transpose()


categorical_counts_df.loc['Mode'] = mode_value


categorical_counts_df


In [ ]:
sum_of_counts = value_counts.sum()
sum_of_counts

In [ ]:
df

In [ ]:
#log scale graph of count of locations
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.figure(figsize=(15,5))
city_counts = df['Location'].value_counts().reset_index()
city_counts.columns = ['Location', 'count']


city_counts['log_count'] = np.log1p(city_counts['count'])

sns.barplot(x='Location', y='log_count', data=city_counts, color='grey')
plt.ylabel('Log(Count)')
plt.title('Log Counts of Cities')
plt.xticks(rotation=90)
plt.show()

In [ ]:
##log scale graph of count of locations but where cities with 1 count are aggregated in an other set.
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

city_counts = df['Location'].value_counts().reset_index()
city_counts.columns = ['Location', 'count']

city_counts['Location'] = city_counts.apply(
    lambda row: 'Other' if row['count'] == 1 else row['Location'], axis=1)


city_counts = city_counts.groupby('Location').sum().reset_index()


city_counts['log_count'] = np.log1p(city_counts['count'])

plt.figure(figsize=(15,5))
sns.barplot(x='Location', y='log_count', data=city_counts, color='grey')
plt.ylabel('Log(Count)')
plt.title('Log Counts of Cities (with "Other" Category)')
plt.xticks(rotation=90)
plt.show()


##Branch

In [ ]:
#count of branch sector
import pandas as pd

categorical_column = 'Branch Sector'

mode_value = df[categorical_column].mode()[0]


value_counts = df[categorical_column].value_counts()


total_counts = df[categorical_column].count()
percentages = (value_counts / total_counts) * 100


categorical_counts_df = pd.DataFrame({'Counts': value_counts, 'Percentages': percentages})


categorical_counts_df = categorical_counts_df.transpose()


categorical_counts_df.loc['Mode'] = mode_value


categorical_counts_df

In [ ]:
#log scale graph of count of branches
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.figure(figsize=(15,5))
city_counts = df['Branch Sector'].value_counts().reset_index()
city_counts.columns = ['Branch Sector', 'count']


city_counts['log_count'] = np.log1p(city_counts['count'])

sns.barplot(x='Branch Sector', y='log_count', data=city_counts, color='grey')
plt.ylabel('Log(Count)')
plt.title('Log Counts of Branch Sector')
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Mean of each branch sector
import matplotlib.pyplot as plt
import seaborn as sns

columns_to_plot = ['Net Sales']

for col in columns_to_plot:
    if col in df.columns:
        sector_means = df.groupby('Branch Sector')[col].mean().reset_index()

        plt.figure(figsize=(15,5))
        sns.barplot(x='Branch Sector', y=col, data=sector_means, palette=['grey'])
        plt.ylabel(col)
        plt.title(f'Mean of {col} by Branch Sector')
        plt.xticks(rotation=90)
        plt.show()


In [ ]:
#log scale graph of average of net sales.
import matplotlib.pyplot as plt
import seaborn as sns
df['log_avg'] = np.log1p(df['avg'])
columns_to_plot = ['log_avg']

for col in columns_to_plot:
    if col in df.columns:
        sector_means = df.groupby('Branch Sector')[col].mean().reset_index()

        plt.figure(figsize=(15,5))
        sns.barplot(x='Branch Sector', y='log_avg', data=sector_means, palette=['grey'])  # Setting the color to black
        plt.ylabel(col)
        plt.title(f'Log of Average Net Sales by Branch Sector')
        plt.xticks(rotation=90)
        plt.show()

In [ ]:
# Average net sales per branch sector per Final second round year
import matplotlib.pyplot as plt
import seaborn as sns

bya = df[['Branch Sector','Final Second Round Year','avg']]


# Creating the plot
plt.figure(figsize=(15,7))
sns.barplot(x='Branch Sector', y='avg', hue='Final Second Round Year', data=bya)
plt.title('Average Value by Branch Sector and Final Second Round Year')
plt.xticks(rotation=90)
plt.ylabel('Average Value')
plt.xlabel('Branch Sector')
plt.legend(title='Final Second Round Year', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
# Logarithmic scale of previous graph
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


bya['log_avg'] = np.log1p(bya['avg'])

plt.figure(figsize=(15,7))
sns.barplot(x='Branch Sector', y='log_avg', hue='Final Second Round Year', data=bya)
plt.title('Logarithmic Average Net sales by Branch Sector and Final Second Round Year')
plt.xticks(rotation=90)
plt.ylabel('Log(Average Net Sales)')
plt.xlabel('Branch Sector')
plt.legend(title='Final Second Round Year', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
#average net sales per branch peer year

pivot_table = bya.pivot_table(index='Branch Sector',
                              columns='Final Second Round Year',
                              values='avg',
                              aggfunc='mean')

pivot_table


In [ ]:
#counts of success and failue
import pandas as pd

categorical_column = 'More than one round'

mode_value = df[categorical_column].mode()[0]


value_counts = df[categorical_column].value_counts()


total_counts = df[categorical_column].count()
percentages = (value_counts / total_counts) * 100


categorical_counts_df = pd.DataFrame({'Counts': value_counts, 'Percentages': percentages})


categorical_counts_df = categorical_counts_df.transpose()


categorical_counts_df.loc['Mode'] = mode_value


categorical_counts_df

In [ ]:
import pandas as pd

categorical_column = 'Founded on Year'

mode_value = df[categorical_column].mode()[0]


value_counts = df[categorical_column].value_counts()


total_counts = df[categorical_column].count()
percentages = (value_counts / total_counts) * 100


categorical_counts_df = pd.DataFrame({'Counts': value_counts, 'Percentages': percentages})


categorical_counts_df = categorical_counts_df.transpose()


categorical_counts_df.loc['Mode'] = mode_value


categorical_counts_df

##final second round

In [ ]:
#success counts per year
import pandas as pd

categorical_column = 'Final Second Round Year'

mode_value = df[categorical_column].mode()[0]


value_counts = df[categorical_column].value_counts()


total_counts = df[categorical_column].count()
percentages = (value_counts / total_counts) * 100


categorical_counts_df = pd.DataFrame({'Counts': value_counts, 'Percentages': percentages})


categorical_counts_df = categorical_counts_df.transpose()


categorical_counts_df.loc['Mode'] = mode_value


categorical_counts_df

#including gender


This code has the same implications as the above coude so only comments on new code will be supllied

In [ ]:
#Viewing rows with no share of female founders data
rows_with_na = df[df['Share female founders'].isna()]
rows_with_na

In [ ]:
#removing rows with no share of female founders data
df = df.dropna(subset=['Share female founders'])
df

 If one filters with the data with and without the gender filtration the same outliers and number fo outliers are removed so no need to conduct further filtering.


In [ ]:
df.describe()


In [ ]:
df.dtypes

In [ ]:
import pandas as pd

non_numerical_columns = ['x', 'Name', 'Location']

summary_df = pd.DataFrame(columns=['Column', 'Mode', 'Median', 'Mean', 'Q1', 'Q2', 'Q3', 'Q4'])

for column in df.columns:
    if column not in non_numerical_columns:

        mode = df[column].mode()[0]
        median = df[column].median()
        mean = df[column].mean()
        std=df[column].std()
        q1 = df[column].quantile(0.25)
        q2 = df[column].quantile(0.50)
        q3 = df[column].quantile(0.75)
        q4 = df[column].quantile(1.00)


        summary_df = summary_df.append({
            'Column': column,
            'Mode': mode,
            'Median': median,
            'Mean': mean,
            'Stan dev.': std,
            'Q1': q1,
            'Q2': q2,
            'Q3': q3,
            'Q4': q4
        }, ignore_index=True)


summary_df


In [ ]:
import pandas as pd

categorical_column = 'Location'

mode_value = df[categorical_column].mode()[0]


value_counts = df[categorical_column].value_counts()


total_counts = df[categorical_column].count()
percentages = (value_counts / total_counts) * 100


categorical_counts_df = pd.DataFrame({'Counts': value_counts, 'Percentages': percentages})


categorical_counts_df = categorical_counts_df.transpose()


categorical_counts_df.loc['Mode'] = mode_value


categorical_counts_df


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.figure(figsize=(15,5))
city_counts = df['Location'].value_counts().reset_index()
city_counts.columns = ['Location', 'count']


city_counts['log_count'] = np.log1p(city_counts['count'])

sns.barplot(x='Location', y='log_count', data=city_counts, color='grey')
plt.ylabel('Log(Count)')
plt.title('Log Counts of Cities')
plt.xticks(rotation=90)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming df is your DataFrame
# df = pd.read_csv('your_dataframe.csv')  # Replace with your data loading code

# Count the occurrences of each location
city_counts = df['Location'].value_counts().reset_index()
city_counts.columns = ['Location', 'count']

# Aggregate locations with a count of 1 into 'Other'
city_counts['Location'] = city_counts.apply(
    lambda row: 'Other' if row['count'] == 1 else row['Location'], axis=1)

# Recalculate the counts (including the new 'Other' category)
city_counts = city_counts.groupby('Location').sum().reset_index()

# Calculating the log of the count
city_counts['log_count'] = np.log1p(city_counts['count'])

# Plotting
plt.figure(figsize=(15,5))
sns.barplot(x='Location', y='log_count', data=city_counts, color='grey')
plt.ylabel('Log(Count)')
plt.title('Log Counts of Cities (with "Other" Category)')
plt.xticks(rotation=90)
plt.show()


##Branch

In [ ]:
import pandas as pd

categorical_column = 'Branch Sector'

mode_value = df[categorical_column].mode()[0]


value_counts = df[categorical_column].value_counts()


total_counts = df[categorical_column].count()
percentages = (value_counts / total_counts) * 100


categorical_counts_df = pd.DataFrame({'Counts': value_counts, 'Percentages': percentages})


categorical_counts_df = categorical_counts_df.transpose()


categorical_counts_df.loc['Mode'] = mode_value


categorical_counts_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.figure(figsize=(15,5))
city_counts = df['Branch Sector'].value_counts().reset_index()
city_counts.columns = ['Branch Sector', 'count']


city_counts['log_count'] = np.log1p(city_counts['count'])

sns.barplot(x='Branch Sector', y='log_count', data=city_counts, color='grey')
plt.ylabel('Log(Count)')
plt.title('Log Counts of Branch Sector')
plt.xticks(rotation=90)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

columns_to_plot = ['Net Sales']

for col in columns_to_plot:
    if col in df.columns:
        sector_means = df.groupby('Branch Sector')[col].mean().reset_index()

        plt.figure(figsize=(15,5))
        sns.barplot(x='Branch Sector', y=col, data=sector_means, palette=['grey'])
        plt.ylabel(col)
        plt.title(f'Mean of {col} by Branch Sector')
        plt.xticks(rotation=90)
        plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
df['log_avg'] = np.log1p(df['avg'])
columns_to_plot = ['log_avg']

for col in columns_to_plot:
    if col in df.columns:
        sector_means = df.groupby('Branch Sector')[col].mean().reset_index()

        plt.figure(figsize=(15,5))
        sns.barplot(x='Branch Sector', y='log_avg', data=sector_means, palette=['grey'])  # Setting the color to black
        plt.ylabel('Log Average')
        plt.title(f'Log of Average Net Sales by Branch Sector')
        plt.xticks(rotation=90)
        plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

bya = df[['Branch Sector','Final Second Round Year','avg']]


# Creating the plot
plt.figure(figsize=(15,7))
sns.barplot(x='Branch Sector', y='avg', hue='Final Second Round Year', data=bya)
plt.title('Average Value by Branch Sector and Final Second Round Year')
plt.xticks(rotation=90)
plt.ylabel('Average Value')
plt.xlabel('Branch Sector')
plt.legend(title='Final Second Round Year', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


bya['log_avg'] = np.log1p(bya['avg'])

plt.figure(figsize=(15,7))
sns.barplot(x='Branch Sector', y='log_avg', hue='Final Second Round Year', data=bya)
plt.title('Logarithmic Average Net sales by Branch Sector and Final Second Round Year')
plt.xticks(rotation=90)
plt.ylabel('Log(Average Net Sales)')
plt.xlabel('Branch Sector')
plt.legend(title='Final Second Round Year', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:


pivot_table = bya.pivot_table(index='Branch Sector',
                              columns='Final Second Round Year',
                              values='avg',
                              aggfunc='mean')

pivot_table


##more than one round

In [ ]:
import pandas as pd

categorical_column = 'More than one round'

mode_value = df[categorical_column].mode()[0]


value_counts = df[categorical_column].value_counts()


total_counts = df[categorical_column].count()
percentages = (value_counts / total_counts) * 100


categorical_counts_df = pd.DataFrame({'Counts': value_counts, 'Percentages': percentages})


categorical_counts_df = categorical_counts_df.transpose()


categorical_counts_df.loc['Mode'] = mode_value


categorical_counts_df

In [ ]:
import pandas as pd

categorical_column = 'Founded on Year'

mode_value = df[categorical_column].mode()[0]


value_counts = df[categorical_column].value_counts()


total_counts = df[categorical_column].count()
percentages = (value_counts / total_counts) * 100


categorical_counts_df = pd.DataFrame({'Counts': value_counts, 'Percentages': percentages})


categorical_counts_df = categorical_counts_df.transpose()


categorical_counts_df.loc['Mode'] = mode_value


categorical_counts_df

##final second round

In [ ]:
import pandas as pd

categorical_column = 'Final Second Round Year'

mode_value = df[categorical_column].mode()[0]


value_counts = df[categorical_column].value_counts()


total_counts = df[categorical_column].count()
percentages = (value_counts / total_counts) * 100


categorical_counts_df = pd.DataFrame({'Counts': value_counts, 'Percentages': percentages})


categorical_counts_df = categorical_counts_df.transpose()


categorical_counts_df.loc['Mode'] = mode_value


categorical_counts_df